In [7]:
# manipulation des données
import numpy as np
import pandas as pd

# matplotlib et seaborn pour les représentations graphiques
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# sklearn preprocessing pour le traiter les variables catégorielles
from sklearn.preprocessing import LabelEncoder

# Gestion du système de fichiers
import os

# Suppression des alertes
import warnings
warnings.filterwarnings('ignore')

In [8]:
df = pd.read_csv("../Data/2.sample_dataset.csv")
df = df.dropna(subset=['words'])
df = df.sample(1000, random_state=42)
df

,target,id,date,flag,user,text,words
486066,1,1564732376,Mon Apr 20 04:43:34 PDT 2009,NO_QUERY,john1954moi,hey no problem like getting followers and im p...,hey problem like get follower im pleased follo...
834217,1,2059479762,Sat Jun 06 17:01:07 PDT 2009,NO_QUERY,Hinn888,thanks,thanks
844489,0,1996685745,Mon Jun 01 15:30:36 PDT 2009,NO_QUERY,queenbmakeup,have this nail strengther from them but nothi...,nail strengther nothing work nail
179878,1,1967004216,Fri May 29 18:47:52 PDT 2009,NO_QUERY,angelope,waiting for the decemberists,wait decemberists
244128,1,1976584126,Sat May 30 18:30:30 PDT 2009,NO_QUERY,wbgookin,herb tarlek on wkrp could but only if he wore ...,herb tarlek wkrp could wore white shoe
...,...,...,...,...,...,...,...
922825,1,2002545720,Tue Jun 02 04:31:18 PDT 2009,NO_QUERY,iLoveFry_,my personal feeling is that everyone should go...,personal feel everyone go buy paolo nutinis ne...
973226,0,2068855166,Sun Jun 07 14:49:21 PDT 2009,NO_QUERY,JamieLeann215,just finished helping move buncha stuff now im...,finish help move buncha stuff im wait lay neig...
615817,1,1824267208,Sun May 17 00:40:20 PDT 2009,NO_QUERY,JoeRialiS,omg finished from see your video quotheyquot i...,omg finish see video quotheyquot amaze funny l...
107004,1,1833727993,Mon May 18 00:14:56 PDT 2009,NO_QUERY,jeffwidman,great dinner and crew memorable momentwhen six...,great dinner crew memorable momentwhen six peo...


In [15]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

In [18]:
X = df['words']
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(800,)
(800,)
(200,)
(200,)


In [19]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
def tokenize_data(data, tokenizer, max_length=128):
    tokenized_data = tokenizer(data.tolist(), max_length=max_length, padding=True, truncation=True, return_tensors="pt")
    return tokenized_data

train_tokenized = tokenize_data(X_train, tokenizer)
test_tokenized = tokenize_data(X_test, tokenizer)

In [23]:
train_data = TensorDataset(train_tokenized['input_ids'], train_tokenized['attention_mask'], torch.tensor(y_train.tolist()))
test_data = TensorDataset(test_tokenized['input_ids'], test_tokenized['attention_mask'], torch.tensor(y_test.tolist()))

batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

epochs = 3

for epoch in range(epochs):
    model.train()
    for input_ids, attention_mask, labels in train_loader:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the fine-tuned model
#model.save_pretrained("path/to/save/fine-tuned-model")

In [25]:
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for input_ids, attention_mask, labels in test_loader:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(predicted_labels)
        true_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.5950
